# Python a C

V pythonu je více možnjostí, jak volat C funkce z pythoního kódu (modul `ctypes`, SWIG, python/C API).

**Proč to dělat?** Rychlost (když pypy nebo cython nestačí), potřeba použít existující obskurní či osvědčenou knihovnu (\*.SO/\*.DLL), *protože proto*! ;-)

## Zdroje k studiu:

*   https://docs.python.org/3.8/extending/extending.html
*   https://www.journaldev.com/31907/calling-c-functions-from-python
*   https://book.pythontips.com/en/latest/python_c_extension.html (odtud pochází příklad pro Python/C API)

---


# ctypes

In [2]:
%%writefile mylib.c
#include <stdio.h>
#include <math.h>
int rnd();
int add(int, int);
float fadd(float, float);
float fsqrt(float);
double PI = 3.14;

int rnd() {
	return 9;  //# https://dilbert.com/strip/2001-10-25
}

int add(int a, int b) {
	return a + b;
}

float fadd(float a, float b) {
	return a + b;
}

float fsqrt(float a) {
	return sqrt(a);
}

Overwriting mylib.c


In [3]:
!rm mylib.so 2>&1 > /dev/null
# -fPIC ... https://gcc.gnu.org/onlinedocs/gcc/Code-Gen-Options.html#Code-Gen-Options
!gcc -fPIC -shared -Wl,-soname,mylib -o mylib.so mylib.c
!ls -la

rm: cannot remove 'mylib.so': No such file or directory
total 28
drwxr-xr-x 1 root root 4096 May  5 14:21 .
drwxr-xr-x 1 root root 4096 May  5 14:05 ..
drwxr-xr-x 4 root root 4096 May  3 13:41 .config
-rw-r--r-- 1 root root  332 May  5 14:20 mylib.c
-rwxr-xr-x 1 root root 7680 May  5 14:21 mylib.so
drwxr-xr-x 1 root root 4096 May  3 13:42 sample_data


In [7]:
import ctypes
from ctypes import *

# raloading magic - https://stackoverflow.com/a/62021495
def ctypesCloseLibrary(lib):
    dlclose_func = ctypes.CDLL(None).dlclose
    dlclose_func.argtypes = [ctypes.c_void_p]
    dlclose_func.restype = ctypes.c_int
    dlclose_func(lib._handle)

ml = CDLL("./mylib.so")
#ml.fsqrt.argtypes = (c_float,)
ml.fsqrt.restype = c_float
ml.fadd.argtypes = (c_float, c_float)
ml.fadd.restype = c_float

print(type(ml))
print(f'random number by C code = {ml.rnd(10)}')
print(f'ints A + B by C code = {ml.add(22, 20)}')
print(f'floats A + B by C code = {ml.fadd(22, 20.0)}')  # nekompatibilni typy! -> nutno uvest pomoci: ml.fadd.argtypes = (c_float, c_float)
print(f'floats A + B by C code = {ml.fadd(c_float(22.0), c_float(20))}')  
print(f'float sqrt = {ml.fsqrt(c_float(25.0))}')  

ctypesCloseLibrary(ml)

<class 'ctypes.CDLL'>
random number by C code = 9
ints A + B by C code = 42
floats A + B by C code = 42.0
floats A + B by C code = 42.0
float sqrt = 5.0


# SWIG

SWIG = Simplified Wrapper and Interface Generator, se nejspíše vyplatí použít, pokud chcete svůj C/C++ kód zpřístupňovat ve více jazycích. Jednou napíšete interface a pak už generujete wrappery.

In [8]:
!apt-get install swig python3.7-dev libpython3.7-dev
!python --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libpython3.7-dev is already the newest version (3.7.13-1+bionic3).
libpython3.7-dev set to manually installed.
python3.7-dev is already the newest version (3.7.13-1+bionic3).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100

In [9]:
%%writefile mylib2.i

/* mylib2.i */
 %module mylib2
 %{
 /* Put header files here or function declarations like below */
 extern double PI;
 extern int rnd();
 extern int add(int a, int b);
 extern float fadd(float a, float b);
 %}
 
 extern double PI;
 extern int rnd();
 extern int add(int a, int b);
 extern float fadd(float a, float b);

Writing mylib2.i


In [10]:
#!rm ./mylib*
!swig -python mylib2.i
!ls -la

total 152
drwxr-xr-x 1 root root   4096 May  5 14:28 .
drwxr-xr-x 1 root root   4096 May  5 14:05 ..
drwxr-xr-x 4 root root   4096 May  3 13:41 .config
-rw-r--r-- 1 root root    320 May  5 14:28 mylib2.i
-rw-r--r-- 1 root root   3143 May  5 14:28 mylib2.py
-rw-r--r-- 1 root root 118379 May  5 14:28 mylib2_wrap.c
-rw-r--r-- 1 root root    332 May  5 14:20 mylib.c
-rwxr-xr-x 1 root root   7680 May  5 14:21 mylib.so
drwxr-xr-x 1 root root   4096 May  3 13:42 sample_data


Přibilo **mylib.py**, podíváme se na něj:

In [11]:
!cat mylib2.py

# This file was automatically generated by SWIG (http://www.swig.org).
# Version 3.0.12
#
# Do not make changes to this file unless you know what you are doing--modify
# the SWIG interface file instead.

from sys import version_info as _swig_python_version_info
if _swig_python_version_info >= (2, 7, 0):
    def swig_import_helper():
        import importlib
        pkg = __name__.rpartition('.')[0]
        mname = '.'.join((pkg, '_mylib2')).lstrip('.')
        try:
            return importlib.import_module(mname)
        except ImportError:
            return importlib.import_module('_mylib2')
    _mylib2 = swig_import_helper()
    del swig_import_helper
elif _swig_python_version_info >= (2, 6, 0):
    def swig_import_helper():
        from os.path import dirname
        import imp
        fp = None
        try:
            fp, pathname, description = imp.find_module('_mylib2', [dirname(__file__)])
        except ImportError:
            import _mylib2
            return _mylib2
     

Vyrobíme **mylib2.so**:

In [12]:
!gcc -fPIC -c mylib.c mylib2_wrap.c -I/usr/include/python3.7m/
!ld -shared mylib.o mylib2_wrap.o -o _mylib2.so
!ls -la
#!cat mylib2_wrap.c

total 264
drwxr-xr-x 1 root root   4096 May  5 14:30 .
drwxr-xr-x 1 root root   4096 May  5 14:05 ..
drwxr-xr-x 4 root root   4096 May  3 13:41 .config
-rw-r--r-- 1 root root    320 May  5 14:28 mylib2.i
-rw-r--r-- 1 root root   3143 May  5 14:28 mylib2.py
-rwxr-xr-x 1 root root  50752 May  5 14:30 _mylib2.so
-rw-r--r-- 1 root root 118379 May  5 14:28 mylib2_wrap.c
-rw-r--r-- 1 root root  53704 May  5 14:30 mylib2_wrap.o
-rw-r--r-- 1 root root    332 May  5 14:20 mylib.c
-rw-r--r-- 1 root root   1760 May  5 14:30 mylib.o
-rwxr-xr-x 1 root root   7680 May  5 14:21 mylib.so
drwxr-xr-x 1 root root   4096 May  3 13:42 sample_data


A hurá to použít

In [13]:
import mylib2 as ml2
print(f'random number by C code = {ml2.rnd()}')
print(f'ints A + B by C code = {ml2.add(22, 20)}')
print(f'floats A + B by C code = {ml2.fadd(22.0, 20.0)}')

random number by C code = 9
ints A + B by C code = 42
floats A + B by C code = 42.0


# Python/C API

Poslední možnost je asi nejkomplikovanější, ale na druhou stranu nám umožňuje jako jediná z uvedených modifikovat pythonové objekty v C kódu.

Základem je `Python.h` header file, který poskytuje funkce (+/- ekvivalenty k funkcím z `builtins`) k manipulaci se strukturou `PyObject`. Ta reprezentuje libovolný pythonový objekt. Příkladem budiž `PyList_Size()`, která počítá délku `PyListType` (tedy listu).

Napíšeme si rozšíření pythonu v C, které sečte prvky v listu (předpokládáme numerické hodnoty). Začneme s výsledným použitím:

In [14]:
%%writefile test_adder.py
import addList
l = [1, 2, 3, 4, 5]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [1, 2, 3, 4, 5, 'bug', 6, 7]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = ['a', 'b']
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [42, 42**12]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")

Writing test_adder.py


Teď je na řadě samotná implementace v C:

In [15]:
%%writefile adder.c
#define PY_SSIZE_T_CLEAN
#include <Python.h>

// hlavni funkce volana z pythonu, konvece pojmenoveni je:
// {module-name}_{function-name}
static PyObject* addList_add(PyObject* self, PyObject* args){
  return Py_BuildValue("i", 42); // prozatim vratime python objekt integer...
}

static PyMethodDef module_functions[] = {
    {"add", (PyCFunction)addList_add, METH_VARARGS, PyDoc_STR("sums the list")},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

static struct PyModuleDef cAddList =
{
    PyModuleDef_HEAD_INIT,
    "addList",   /* name of module */
    "",          /* module documentation, may be NULL */
    -1,          /* size of per-interpreter state of the module, or -1 if the module keeps state in global variables. */
    module_functions
};

PyMODINIT_FUNC PyInit_addList(void)
{
    return PyModule_Create(&cAddList);
}


Writing adder.c


Jdeme to zkompilovat:

In [16]:
%%writefile setup.py
from distutils.core import setup, Extension
setup(name='addList', version='1.0', ext_modules=[Extension('addList', ['adder.c'])])

Writing setup.py


In [18]:
import sys
import subprocess
!python3 setup.py install
r = subprocess.run([sys.executable, "test_adder.py"], capture_output=True)
print(f"Output:\n{r.stdout.decode('UTF-8')}")

running install
running build
running build_ext
building 'addList' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c adder.c -o build/temp.linux-x86_64-3.7/adder.o
creating build/lib.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 -Wl,-Bsymbolic-functions -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/adder.o -o build/lib.linux-x86_64-3.7/addList.cpython-37m-x86_64-linux-gnu.so
running install_lib
copying build/lib.linux-x86_64-3.7/addList.cpython-37m-x86_64-linux-gnu.so -> /usr/local/lib/python3.7/dist-packages
run

A pojďme doplnit i samotný výkonný kód, který nám posčítá prvky listu:

In [19]:
%%writefile adder.c
#define PY_SSIZE_T_CLEAN
#include <Python.h>

// hlavni funkce volana z pythonu, konvece pojmenoveni je:
// {module-name}_{function-name}
static PyObject* addList_add(PyObject* self, PyObject* args){
  PyObject * listObj;

  //The input arguments come as a tuple, we parse the args to get the various variables
  //In this case it's only one list variable, which will now be referenced by listObj
  if (! PyArg_ParseTuple( args, "O", &listObj))
    return NULL;

  Py_ssize_t i, n;
  n = PyList_Size(listObj);
  if (n < 0)
    Py_RETURN_NONE; // listObj is not a list 

  long sum = 0, elem;
  for(i = 0; i < n; i++){
    PyObject* temp = PyList_GetItem(listObj, i);
    if (!PyLong_Check(temp)) continue; // lets skip non-integers
    elem = PyLong_AsLong(temp);
    // printf("e=%dl", elem);
    if (elem==-1 && PyErr_Occurred())
      Py_RETURN_NONE; // number not fit into C long
    sum += elem;
  }

  // value returned back to python code - another python object
  return Py_BuildValue("i", sum);
}

static PyMethodDef module_functions[] = {
    {"add", (PyCFunction)addList_add, METH_VARARGS, PyDoc_STR("sums the list")},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

static struct PyModuleDef cAddList =
{
    PyModuleDef_HEAD_INIT,
    "addList",   /* name of module */
    "",          /* module documentation, may be NULL */
    -1,          /* size of per-interpreter state of the module, or -1 if the module keeps state in global variables. */
    module_functions
};

PyMODINIT_FUNC PyInit_addList(void)
{
    return PyModule_Create(&cAddList);
}

Overwriting adder.c


In [20]:
%%writefile test_adder.py
import addList
l = [1, 2, 3, 4, 5]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [1, 2, 3, 4, 5, 'bug', 6, 7]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = ['a', 'b']
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [42, 42**12]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")


Overwriting test_adder.py


In [21]:
import sys
import subprocess
!python3 setup.py install
r = subprocess.run([sys.executable, "test_adder.py"], capture_output=True)
print(r)
print(f"Output:\n{r.stdout.decode('UTF-8')}")
print(f"Error:\n{r.stderr.decode('UTF-8')}" if len(r.stderr) > 0 else '') 

running install
running build
running build_ext
building 'addList' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c adder.c -o build/temp.linux-x86_64-3.7/adder.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 -Wl,-Bsymbolic-functions -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/adder.o -o build/lib.linux-x86_64-3.7/addList.cpython-37m-x86_64-linux-gnu.so
running install_lib
copying build/lib.linux-x86_64-3.7/addList.cpython-37m-x86_64-linux-gnu.so -> /usr/local/lib/python3.7/dist-packages
running install_egg_info
Removing /usr/local/lib/python3.7/dist-packages/addList-1.0.egg-in